# Xóa hết bình luận rỗng

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_json('total_comment(13-4-2022).json')
print(df.head())
df.info()

In [ ]:
# bình luận theo số sao
r0 = df[df['rate'] == 0]
r1 = df[df['rate'] == 1]
r2 = df[df['rate'] == 2]
r3 = df[df['rate'] == 3]
r4 = df[df['rate'] == 4]
r5 = df[df['rate'] == 5]

In [ ]:
# Bình luận rỗng theo số sao
r0_empty = r0[r0['content'] == '']
r1_empty = r1[r1['content'] == '']
r2_empty = r2[r2['content'] == '']
r3_empty = r3[r3['content'] == '']
r4_empty = r4[r4['content'] == '']
r5_empty = r5[r5['content'] == '']

In [ ]:
# Tổng bình luận rỗng
print('số bl rỗng 0s',len(r0_empty))
print('số bl rỗng 1s',len(r1_empty))
print('số bl rỗng 2s',len(r2_empty))
print('số bl rỗng 3s',len(r3_empty))
print('số bl rỗng 4s',len(r4_empty))
print('số bl rỗng 5s',len(r5_empty))

In [ ]:
# Xóa tất cả bl rỗng
df = df.drop(r0_empty.index)
df = df.drop(r1_empty.index)
df = df.drop(r2_empty.index)
df = df.drop(r3_empty.index)
df = df.drop(r4_empty.index)
df = df.drop(r5_empty.index)

# NLP processing


In [ ]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import os
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import re
from unicodedata import normalize
import requests
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import KeyedVectors 


In [ ]:
X1 = df['content']
y = df['rate']

In [ ]:
# Chuẩn hóa: thay thế từ viết tắt và xóa các từ dừng 
td = open('tu_dung', 'r', encoding='UTF-8').readlines()
vt = open('viet_tat', 'r', encoding='UTF-8').readlines()

In [ ]:
# Chuyển sang viết thường
for e in X1:
    e.lower()

In [ ]:
# Thay thế từ viết tắt
def dong_nghia(self):
    list_text = self.split(' ')
    for i in range(len(list_text)):
        for j in range(len(vt)):
            if(list_text[i] == vt[j][0]):
                list_text[i] = vt[j][1]
    self = ' '.join(list_text)
    return self

In [ ]:
def tu_dung(self):
    words = []
    split_words = self.split()
    for word in split_words:
        if word not in td:
            words.append(word)
    return ' '.join(words) 

In [ ]:
# Xóa từ dừng 
X2 = []
for e in X1:
    X2.append(tu_dung(e))
    

In [ ]:
X3 = []
for e in X2:
    X3.append(dong_nghia(e))

In [ ]:
# Xóa ký tự đặc biệt và tách từ
X = []
for e in X3:
    lines = gensim.utils.simple_preprocess(e)
    lines = ' '.join(lines)
    lines = ViTokenizer.tokenize(lines)
    X.append(lines)


In [ ]:
import pickle
# File pickle chứa cmt đã đc xóa ký tự đb và tt
pickle.dump(X, open('X_data.pkl', 'wb'))
pickle.dump(y, open('y_data.pkl', 'wb'))

In [ ]:
# Chia tập dữ liệu để kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Vectorizer:Tạo 1 ma trận mà mỗi hàng sẽ đại diện cho một văn bản, mỗi cột đại diện cho 
# một từ có trong từ điển, và mỗi ô (cell) sẽ chứa tần suất xuất hiện của từ trong văn bản tương ứng.
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_train)

# transform the training and validation train using count vectorizer object
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)
X_train_count

In [ ]:
# Word embeded
from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "tii_cmt_process/vi/vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = list(w2v.index_to_key)
wv = w2v

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                sentence.append(wv[word])

        word2vec_data.append(sentence)

    return word2vec_data

X_data_w2v = get_word2vec_data(X)
X_test_w2v = get_word2vec_data(X_test)